## grpc服务端

本章的主要内容是制作运行于windows上的rpc服务端。

##### 环境介绍
使用python虚拟环境，使用步骤如下：
1. 按键win+R，输入cmd

2. 创建python虚拟环境
  python3 -m venv venv --prompt rpc_server

3. 激活python虚拟环境
  venv\Scripts\activate

4. 安装依赖
  pip install grpcio grpcio-tools plyer pyinstaller

##### rpc协议编写

1） 制作rpc协议文件
执行  notepad StockNotify.proto

然后粘贴如下内容：
```
syntax = "proto3";

service Greeter {
  // 定义一个简单的 RPC 方法
  rpc StockNotify (StockRequest) returns (StockReply);
}

// 定义请求和响应消息
message StockRequest {
  string stock_code = 1;
  string stocck_name = 2;
  double latest_price= 3;
  string message = 4;
}

message StockReply {
  int32 code = 1;
  string body = 2;
}
```

2)  执行如下命令，生产rpc协议文件：

```
python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. StockNotify.proto
```


##### rpc服务端代码
执行  notepad stock_notify.py

粘贴如下内容：
```python
import grpc
from concurrent import futures
import time
from plyer import notification


import StockNotify_pb2
import StockNotify_pb2_grpc

# 实现 Greeter 服务
class GreeterServicer(StockNotify_pb2_grpc.GreeterServicer):
    def StockNotify(self, request, context):
        stock_code = request.stock_code
        stock_name = request.stocck_name
        latest_price = request.latest_price
        message = request.message

        notify_title = f"股票{stock_name}当前价格{latest_price}"
        notify_msg = f"{message}"

        notification.notify(title=notify_title,message=notify_msg,timeout=10)
        return StockNotify_pb2.StockReply(code=0, body="已发出通知")

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))

    StockNotify_pb2_grpc.add_GreeterServicer_to_server(GreeterServicer(), server)

    listen_port = 50051
    server.add_insecure_port(f'0.0.0.0:{listen_port}')

    print(f"Server starting, listen on {listen_port} ...")
    server.start()

    try:
        while True:
            time.sleep(86400)  # 让服务器持续运行
    except KeyboardInterrupt:
        server.stop(0)

if __name__ == '__main__':
    serve()
```

##### 制作exe文件

当前目录下的文件内容如下，执行dir，内如如下：
```
2025/03/05  22:16    <DIR>          .
2025/03/05  21:41    <DIR>          ..
2025/03/05  21:54               367 StockNotify.proto
2025/03/05  21:54             1,744 StockNotify_pb2.py
2025/03/05  21:54             3,355 StockNotify_pb2_grpc.py
2025/03/05  22:16             1,234 stock_notify.py
2025/03/05  21:42    <DIR>          venv
               4 个文件          6,700 字节
```
包含rpc相关的三个文件： StockNotify.proto   StockNotify_pb2.py   StockNotify_pb2_grpc.py
以及一个服务段文件： stock_notify.py

执行如下命令，生产exe文件：

pyinstaller --onefile --hidden-import=plyer.platforms.win.notification stock_notify.py

命令行结束后，可以在dist目录下找到stock_notify.exe文件，鼠标双击启动服务端

### 客户端测试代码
# grpc客户端代码

执行  notepad test_rpc.py

粘贴如下内容：
```python
import grpc
import StockNotify_pb2
import StockNotify_pb2_grpc

def stock_notify(stock_code, stocck_name, latest_price, message="没啥事"):
    channel = grpc.insecure_channel('127.0.0.1:50051')
    stub = StockNotify_pb2_grpc.GreeterStub(channel)
    
    request = StockNotify_pb2.StockRequest(stock_code=stock_code, 
                                           stocck_name=stocck_name,
                                           latest_price=latest_price,
                                           message=message)
    
    response = stub.StockNotify(request)
    print(f"gRPC 调用结果： {response.body}")

stock_notify(stock_code="xxx", stocck_name="xxxx", latest_price=9.0)
```
执行 python test_rpc.py